In [2]:
%load_ext autoreload
%autoreload 2

import sys
import yaml
import json
import numpy as np
import pandas as pd
from pytorch_lightning import Trainer

sys.path.append('..')
sys.path.append('FrustraSeq')
from FrustraSeq.models.FrustraSeq import FrustraSeq
from FrustraSeq.dataloader import FrustrationDataModule

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# general info for script

## 1) Load Data
First we load a simple fasta file containing the headers/ids and their sequences and store it in a dictionary of the form {id: sequence}

In [3]:
fasta_file_path = "../data/frustration/bonomi_ensembles_sequences.fasta"

In [4]:
seqs = {}
with open(fasta_file_path, 'r') as f:
    fasta_data = f.read()
    for line in fasta_data.splitlines():
        if line.startswith(">"):
            header = line[1:]
            seqs[header] = ""
        else:
            seqs[header] += line.strip()
seqs

{'Alb3-A3CT': 'MDENASKIISAGRAKRSIAQPDDAGERFRQLKEQEKRSKKNKAVAKDTVELVEESQSESEEGSDDEEEEAREGALASSTTSKPLPEVGQRRSKRSKRKRTV',
 'FCP1': 'PGPEEQEEEPQPRKPGTRRERTLGAPASSERSAAGGRGPRGHKRKLNEEDAASESSRESSNEDEGSSSEADEMAKALEAELNDLM',
 'emerin_67-170': 'GTRGDADMYDLPKKEDALLYQSKGYNDDYYEESYFTTRTYGEPESAGPSRAVRQSVTSFPDADAFHHQVHDDDLLSSSEEECKDRERPMYGRDSAYQSITHYRPV',
 'UBact': 'MIQSLMPERRERPGDPMPKSPSPLEEGGGPRRPETGSPDKDSLLKRMRRVDPKQAERYRQRTGE',
 'Nsp2_CtlIDR': 'KEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGT',
 'NHE1': 'MINNYLTVPAHKLDSPTMSRARIGSDPLAYEPKEDLPVITIDPASPQSPESVDLVNEELKGKVLGLSRDPAKVAEEDEDDDGGIMMRSKETSSPGTDDVFTPAPSDSPSSQRIQRCLSDP',
 'p61_Hck': 'GGRSSCEDPGCPRDEERAPRMGCMKSKFLQVGGNTFSKTETSASPHCPVYVPDPTSTIKPGPNSHNSNTPGIREAGSE',
 'ACTR': 'GTQNRPLLRNSLDDLVGPPSNLEGQSDERALLDQLHTLLSNTDATGLEEIDRALGIPELVNQGQALEPKQD',
 'Hug1': 'AMADPMTMDQGLNPKQFFLDDVVLQDTLCSMSNRVNKSVKTGYLFPKDHVPSANIIAVERRGGLSDIGKNTSN',
 'PaaA2': 'MDYKDDDDKNRALSPMVSEFETIEQENSYNEWLRAKVATSLADPRPAIPHDEVERRMAERFAKMRKERSKQ',
 'Nt-SOCS5': 'RSLRQRLQDTVGLCFPM

In [5]:
df = pd.DataFrame.from_dict(seqs, orient='index', columns=["sequence"]).reset_index().rename(columns={'index':'id'})

In [6]:
df.head()

,id,sequence
0,Alb3-A3CT,MDENASKIISAGRAKRSIAQPDDAGERFRQLKEQEKRSKKNKAVAK...
1,FCP1,PGPEEQEEEPQPRKPGTRRERTLGAPASSERSAAGGRGPRGHKRKL...
2,emerin_67-170,GTRGDADMYDLPKKEDALLYQSKGYNDDYYEESYFTTRTYGEPESA...
3,UBact,MIQSLMPERRERPGDPMPKSPSPLEEGGGPRRPETGSPDKDSLLKR...
4,Nsp2_CtlIDR,KEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGT


## 2) Load Model
We then load the checkpoint of our trained model and create a dataloader from the dictionary. Config and model weights can be found in the OneDrive folder (TODO update later with HF when published).

In [10]:
# load config.yaml corresponding to the model to be used.
with open(f"../data/it5_ABL_protT5_LORA/config.yaml", 'r') as f:
    config = yaml.safe_load(f)
config["experiment_name"]

'it5_ABL_protT5_LORA'

In [11]:
# either provide path to pretrained model weights or 
# set to huggingface model name (e.g. "Rostlab/prot_t5_xl_uniref50" for protT5) 
config["pLM_model"] = "Rostlab/prot_t5_xl_uniref50"

In [12]:
model = FrustraSeq.load_from_checkpoint(checkpoint_path=f"../data/{config['experiment_name']}/best_val_model.ckpt",
                                        config=config)

Using LoRA fine-tuning for ['q', 'k', 'v', 'o'] layers
trainable params: 1,967,104 || all params: 1,210,107,904 || trainable%: 0.1626
RANK -1: Model initialized.


In [13]:
#adjust dataloader parameters as needed. setting max_seq_length to the max length in the dataframe to avoid truncation.
predict_dataloader = FrustrationDataModule(df=df,
                                            max_seq_length=df["sequence"].str.len().max(), 
                                            batch_size=5,
                                            num_workers=1,
                                            persistent_workers=True,)

In [14]:
# adding the surprisal/discovery dictionary which is used to compute the surprisal feature during inference based on 
# precomputed values (for each aa) from the train set (found in onedrive as well)
with open('../data/frustration/reg_heuristic.json', 'r') as f:
    model.surprisal_dict = json.load(f)
model.surprisal_dict["A"]

{'mean': 0.24633155516241328, 'std': 0.5921655729624687}

## 3) Inference

Lets define a Lightning trainer and run inference (prediction)

In [15]:
trainer = Trainer(accelerator='mps',) # use 'gpu' instead of 'mps' on cuda enabled devices or 'cpu' for cpu only. mps is for Mac.

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [16]:
trainer.predict(model, predict_dataloader)

Loaded 16 sequences for prediction.
Created test dataset for prediction
Test dataset size: 16 samples


/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Predicting DataLoader 0:  25%|██▌       | 1/4 [00:04<00:14,  0.21it/s]

/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/pytorch_lightning/loops/prediction_loop.py:257: predict returned None if it was on purpose, ignore this warning...


Predicting DataLoader 0: 100%|██████████| 4/4 [00:13<00:00,  0.29it/s]


[None, None, None, None]

In [17]:
# model.pred_list contains a list of prediction dictionaries (one per protein)
len(model.pred_list), model.pred_list[0].keys()

(16,
 dict_keys(['residue', 'frustration_index', 'frustration_class', 'entropy', 'surprisal']))

We then can either create protein specific dataframes or one combined one for all protein sequences in the input fasta file.

In [18]:
per_protein_df = pd.DataFrame(model.pred_list[0]) # for the first protein
per_protein_df.head()

,residue,frustration_index,frustration_class,entropy,surprisal
0,M,-0.300976,1,0.862157,-1.705295
1,D,0.093996,1,0.821093,0.657200
2,E,0.624572,2,0.632576,1.546181
3,N,-0.455897,1,0.728476,0.109064
4,A,0.224220,1,0.725651,-0.037340


In [19]:
pred_dfs = []
for pred, id in zip(model.pred_list, df["id"]):
    pred["id"] = id
    pred_dfs.append(pd.DataFrame(pred))
combined_df = pd.concat(pred_dfs, ignore_index=True)
combined_df

,residue,frustration_index,frustration_class,entropy,surprisal,id
0,M,-0.300976,1,0.862157,-1.705295,Alb3-A3CT
1,D,0.093996,1,0.821093,0.657200,Alb3-A3CT
2,E,0.624572,2,0.632576,1.546181,Alb3-A3CT
3,N,-0.455897,1,0.728476,0.109064,Alb3-A3CT
4,A,0.224220,1,0.725651,-0.037340,Alb3-A3CT
...,...,...,...,...,...,...
1332,V,0.647613,2,0.617221,-1.649174,His-PknG_1-75
1333,R,0.028138,1,0.743833,0.095156,His-PknG_1-75
1334,R,0.043294,1,0.765684,0.113401,His-PknG_1-75
1335,L,0.656731,2,0.602273,-1.207965,His-PknG_1-75


In [ ]:
# saving the results to csv files if wanted.
per_protein_df.to_csv("./bonomi_protein1_predictions.csv", index=False)
combined_df.to_csv("./bonomi_all_proteins_predictions.csv", index=False)

Use entropy and surprisal score to filter trough predictions. Below is an example where want to filter for residues in which the model is confident (entropy >= 0.3) but its unlikely to observe this value given the amino acids frustration distribution (-1 >= surprisal score OR surprisal score > 1). A surprisal score of 1 means that the predicted regression value is one standart deviations away from its AA mean. Feel free to play around with both scores :)

In [20]:
combined_df.loc[(combined_df["entropy"] <= 0.4) & ((combined_df["surprisal"] <= -1) | (combined_df["surprisal"] >= 1))]

,residue,frustration_index,frustration_class,entropy,surprisal,id
32,E,1.146633,2,0.343227,2.177570,Alb3-A3CT
36,R,1.218557,2,0.381284,1.528209,Alb3-A3CT
188,R,1.216357,2,0.369156,1.525560,emerin_67-170
268,K,1.313926,2,0.307818,2.044763,emerin_67-170
292,I,0.913598,2,0.177008,-1.189498,UBact
346,E,1.247791,2,0.279468,2.299912,UBact
648,D,-1.551834,0,0.353622,-1.125810,ACTR
748,D,1.263624,2,0.384908,1.924316,PaaA2
779,A,0.944697,2,0.261948,1.179341,PaaA2
808,E,1.356537,2,0.280731,2.431431,PaaA2


In [21]:
# change "R" to any amino acid single letter code to get its surprisal computation parameters
model.surprisal_dict["R"]

{'mean': -0.050906470495787476, 'std': 0.8306870505452515}